In [1]:
%cd '..'

/mnt/scratch/l_singh/hdb/project_dir/histonedb/CURATED_SET


In [2]:
import inspect
import os
import re
import sys

import pandas as pd
from Bio import Entrez, SeqIO
from curated_set_services import CuratedSet

In [3]:
# Load data from histones.csv
curated_set = CuratedSet()
cs = curated_set

cs.data.shape, cs.data.columns

((561, 16),
 Index(['accession', 'type', 'variant_group', 'variant', 'doublet', 'gi',
        'ncbi_gene_id', 'hgnc_gene_name', 'taxonomy_id', 'organism', 'phylum',
        'class', 'taxonomy_group', 'info', 'references', 'sequence'],
       dtype='object'))

In [4]:
my_columns = cs.data.columns

In [5]:
cs.data = cs.data[cs.data["variant_group"] != "gH2B"]

In [6]:
list(cs.has_duplicates())

[]

In [7]:
cs.data[cs.data["accession"] == "CUT18450.1"]

,accession,type,variant_group,variant,doublet,gi,ncbi_gene_id,hgnc_gene_name,taxonomy_id,organism,phylum,class,taxonomy_group,info,references,sequence
CUT18450.1,CUT18450.1,H2B,H2B.S,H2B.S,,,,,1473204,Lilium davidii var. unicolor,Streptophyta,Magnoliopsida,,,32716939,MAPKKKPSKLVGTVTKTRKVTETQTLKVSLTKGLKPEDQQTTTNKF...


In [8]:
cs.data.head()

,accession,type,variant_group,variant,doublet,gi,ncbi_gene_id,hgnc_gene_name,taxonomy_id,organism,phylum,class,taxonomy_group,info,references,sequence
NP_505463.1,NP_505463.1,H2A,cH2A,cH2A_(Animals),,17562014,,,6239,Caenorhabditis elegans,Nematoda,Chromadorea,,,26989147 22650316(?),MSGRGKGGKAKTGGKAKSRSSRAGLQFPVGRLHRILRKGNYAQRVG...
EEC09557.1,EEC09557.1,H2A,cH2A,cH2A_(Animals),,215500063,,,6945,Ixodes scapularis,Arthropoda,Arachnida,,,26989147 22650316(?),MSGRGKGGKVKGKSKTRSSRAGLQFPVGRIHRLLRKGNYAERVGAG...
NP_724343.1,NP_724343.1,H2A,cH2A,cH2A_(Animals),,24585673,,,7227,Drosophila melanogaster,Arthropoda,Insecta,,,26989147 22650316(?),MSGRGKGGKVKGKAKSRSNRAGLQFPVGRIHRLLRKGNYAERVGAG...
XP_001119899.1,XP_001119899.1,H2A,cH2A,cH2A_(Animals),,110764935,,,7460,Apis mellifera,Arthropoda,Insecta,,,26989147 22650316(?),MSGRGKGGKAKAKAKSRSNRAGLQFPVGRIHRLLRKGNYAERVGAG...
EDO48405.1,EDO48405.1,H2A,cH2A,cH2A_(Animals),,156227602,,,45351,Nematostella vectensis,Cnidaria,Anthozoa,,,26989147 22650316(?),MSGRGKGKAKGTKSKTRSSRAGLQFPVGRIHRHLRKGNYAERVGAG...


In [9]:
cs.data.shape

(548, 16)

In [10]:
g_h2b_accessions = [
    "CUT18449.1",
    "CUT18450.1",
    "CUT18445.1",
    "CUT18446.1",
    "CUT18447.1",
    "CUT18448.1",
    "CUT18451.1",
    "CUT18452.1",
]
cs.data.loc[cs.data.index.intersection(g_h2b_accessions), :]

,accession,type,variant_group,variant,doublet,gi,ncbi_gene_id,hgnc_gene_name,taxonomy_id,organism,phylum,class,taxonomy_group,info,references,sequence
CUT18450.1,CUT18450.1,H2B,H2B.S,H2B.S,,,,,1473204,Lilium davidii var. unicolor,Streptophyta,Magnoliopsida,,,32716939,MAPKKKPSKLVGTVTKTRKVTETQTLKVSLTKGLKPEDQQTTTNKF...


In [11]:
df = pd.DataFrame(
    {
        "accession": g_h2b_accessions,
        "type": ["H2B"] * len(g_h2b_accessions),
        "variant_group": ["gH2B"] * len(g_h2b_accessions),
        "variant": ["gH2B"] * len(g_h2b_accessions),
        "references": ["26846354"] * len(g_h2b_accessions),
    }
)
df.index = df.accession
df.at[cs.data.index.intersection(g_h2b_accessions), "references"] = (
    "26846354 "
    + cs.data.loc[cs.data.index.intersection(g_h2b_accessions), "references"]
)
df.shape, df.columns

((8, 5),
 Index(['accession', 'type', 'variant_group', 'variant', 'references'], dtype='object'))

In [12]:
df.head()

,accession,type,variant_group,variant,references
accession,,,,,
CUT18449.1,CUT18449.1,H2B,gH2B,gH2B,26846354
CUT18450.1,CUT18450.1,H2B,gH2B,gH2B,26846354 32716939
CUT18445.1,CUT18445.1,H2B,gH2B,gH2B,26846354
CUT18446.1,CUT18446.1,H2B,gH2B,gH2B,26846354
CUT18447.1,CUT18447.1,H2B,gH2B,gH2B,26846354


In [13]:
cs.data = cs.data[cs.data['accession'] != cs.data.index.intersection(g_h2b_accessions).values[0]]
cs.data.shape

(547, 16)

In [14]:
cs.data[cs.data["accession"] == "CUT18450.1"]

,accession,type,variant_group,variant,doublet,gi,ncbi_gene_id,hgnc_gene_name,taxonomy_id,organism,phylum,class,taxonomy_group,info,references,sequence


In [15]:
cs.save()

cp histones.csv backups/histones.csv-Feb0624193432
Previous data backuped to backups/histones.csv-Feb0624193432
Results saved to histones.csv


In [16]:
cs.data = pd.concat([cs.data, df]).fillna("")
cs.data.shape, list(cs.has_duplicates())

((555, 16), [])

In [17]:
cs.data.tail()

,accession,type,variant_group,variant,doublet,gi,ncbi_gene_id,hgnc_gene_name,taxonomy_id,organism,phylum,class,taxonomy_group,info,references,sequence
CUT18446.1,CUT18446.1,H2B,gH2B,gH2B,,,,,,,,,,,26846354,
CUT18447.1,CUT18447.1,H2B,gH2B,gH2B,,,,,,,,,,,26846354,
CUT18448.1,CUT18448.1,H2B,gH2B,gH2B,,,,,,,,,,,26846354,
CUT18451.1,CUT18451.1,H2B,gH2B,gH2B,,,,,,,,,,,26846354,
CUT18452.1,CUT18452.1,H2B,gH2B,gH2B,,,,,,,,,,,26846354,


In [18]:
cs.data[cs.data["variant"] == "gH2B"].shape

(8, 16)

In [19]:
cs.update_accession_version()
cs.data = cs.data.set_index(cs.data.accession.values)

/home/l_singh/.conda/envs/histdb_env/lib/python3.8/site-packages/Bio/Entrez/__init__.py:658: UserWarning: 
Email address is not specified.

To make use of NCBI's E-utilities, NCBI requires you to specify your
email address with each request.  As an example, if your email address
is A.N.Other@example.com, you can specify it as follows:
   from Bio import Entrez
   Entrez.email = 'A.N.Other@example.com'
In case of excessive usage of the E-utilities, NCBI will attempt to contact
a user at the email address provided before blocking access to the
E-utilities.
  warnings.warn(
/home/l_singh/.conda/envs/histdb_env/lib/python3.8/site-packages/Bio/GenBank/__init__.py:1143: BiopythonParserWarning: Dropping bond qualifier in feature location
  warnings.warn(
/mnt/scratch/l_singh/hdb/project_dir/histonedb/CURATED_SET/curated_set_services.py:267: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See t

In [20]:
cs.data[cs.data["variant"] == "gH2B"].shape

(8, 16)

In [21]:
curated_set.update_taxids(blank_data=True)

Fetched taxid from NCBI 1473204
Fetched taxid from NCBI 1473204
Fetched taxid from NCBI 1473204
Unexpected error: <class 'urllib.error.HTTPError'>, Retrying, attempt 0
Fetched taxid from NCBI 1473204
Fetched taxid from NCBI 1473204
Fetched taxid from NCBI 1473204
Fetched taxid from NCBI 1473204
Fetched taxid from NCBI 1473204
 changes to 1473204
 changes to Lilium davidii var. unicolor
 changes to Streptophyta
 changes to Magnoliopsida
 changes to 1473204
 changes to Lilium davidii var. unicolor
 changes to Streptophyta
 changes to Magnoliopsida
 changes to 1473204
 changes to Lilium davidii var. unicolor
 changes to Streptophyta
 changes to Magnoliopsida
 changes to 1473204
 changes to Lilium davidii var. unicolor
 changes to Streptophyta
 changes to Magnoliopsida
 changes to 1473204
 changes to Lilium davidii var. unicolor
 changes to Streptophyta
 changes to Magnoliopsida
 changes to 1473204
 changes to Lilium davidii var. unicolor
 changes to Streptophyta
 changes to Magnoliopsida


In [22]:
cs.data[cs.data["variant"] == "gH2B"].shape

(8, 16)

In [23]:
cs.data[cs.data["variant"] == "gH2B"]

,accession,type,variant_group,variant,doublet,gi,ncbi_gene_id,hgnc_gene_name,taxonomy_id,organism,phylum,class,taxonomy_group,info,references,sequence
CUT18449.1,CUT18449.1,H2B,gH2B,gH2B,,,,,1473204,Lilium davidii var. unicolor,Streptophyta,Magnoliopsida,,,26846354,
CUT18450.1,CUT18450.1,H2B,gH2B,gH2B,,,,,1473204,Lilium davidii var. unicolor,Streptophyta,Magnoliopsida,,,26846354 32716939,
CUT18445.1,CUT18445.1,H2B,gH2B,gH2B,,,,,1473204,Lilium davidii var. unicolor,Streptophyta,Magnoliopsida,,,26846354,
CUT18446.1,CUT18446.1,H2B,gH2B,gH2B,,,,,1473204,Lilium davidii var. unicolor,Streptophyta,Magnoliopsida,,,26846354,
CUT18447.1,CUT18447.1,H2B,gH2B,gH2B,,,,,1473204,Lilium davidii var. unicolor,Streptophyta,Magnoliopsida,,,26846354,
CUT18448.1,CUT18448.1,H2B,gH2B,gH2B,,,,,1473204,Lilium davidii var. unicolor,Streptophyta,Magnoliopsida,,,26846354,
CUT18451.1,CUT18451.1,H2B,gH2B,gH2B,,,,,1473204,Lilium davidii var. unicolor,Streptophyta,Magnoliopsida,,,26846354,
CUT18452.1,CUT18452.1,H2B,gH2B,gH2B,,,,,1473204,Lilium davidii var. unicolor,Streptophyta,Magnoliopsida,,,26846354,


In [24]:
my_columns == cs.data.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True])

In [25]:
cs.save()

Added sequence with accession CUT18452.1
Added sequence with accession CUT18445.1
Added sequence with accession CUT18448.1
Added sequence with accession CUT18446.1
Added sequence with accession CUT18447.1
Added sequence with accession CUT18451.1
Added sequence with accession CUT18450.1
Added sequence with accession CUT18449.1
cp histones.csv backups/histones.csv-Feb0624193542
Previous data backuped to backups/histones.csv-Feb0624193542
Results saved to histones.csv


In [2]:
!python --version

Python 3.8.1


## Updating sequences for gH2B (added after adding H2B.Z 6-Mar-2024)

In [4]:
cs.data[cs.data['sequence'] == '']

,accession,type,variant_group,variant,doublet,gi,ncbi_gene_id,hgnc_gene_name,taxonomy_id,organism,phylum,class,taxonomy_group,info,references,sequence
CUT18449.1,CUT18449.1,H2B,gH2B,gH2B,,,,,1473204,Lilium davidii var. unicolor,Streptophyta,Magnoliopsida,,,26846354,
CUT18450.1,CUT18450.1,H2B,gH2B,gH2B,,,,,1473204,Lilium davidii var. unicolor,Streptophyta,Magnoliopsida,,,26846354 32716939,
CUT18445.1,CUT18445.1,H2B,gH2B,gH2B,,,,,1473204,Lilium davidii var. unicolor,Streptophyta,Magnoliopsida,,,26846354,
CUT18446.1,CUT18446.1,H2B,gH2B,gH2B,,,,,1473204,Lilium davidii var. unicolor,Streptophyta,Magnoliopsida,,,26846354,
CUT18447.1,CUT18447.1,H2B,gH2B,gH2B,,,,,1473204,Lilium davidii var. unicolor,Streptophyta,Magnoliopsida,,,26846354,
CUT18448.1,CUT18448.1,H2B,gH2B,gH2B,,,,,1473204,Lilium davidii var. unicolor,Streptophyta,Magnoliopsida,,,26846354,
CUT18451.1,CUT18451.1,H2B,gH2B,gH2B,,,,,1473204,Lilium davidii var. unicolor,Streptophyta,Magnoliopsida,,,26846354,
CUT18452.1,CUT18452.1,H2B,gH2B,gH2B,,,,,1473204,Lilium davidii var. unicolor,Streptophyta,Magnoliopsida,,,26846354,


In [5]:
cs.update_sequence(blank_data=True)

Fetching 8 seqs


/home/l_singh/.conda/envs/histdb_env/lib/python3.8/site-packages/Bio/Entrez/__init__.py:658: UserWarning: 
Email address is not specified.

To make use of NCBI's E-utilities, NCBI requires you to specify your
email address with each request.  As an example, if your email address
is A.N.Other@example.com, you can specify it as follows:
   from Bio import Entrez
   Entrez.email = 'A.N.Other@example.com'
In case of excessive usage of the E-utilities, NCBI will attempt to contact
a user at the email address provided before blocking access to the
E-utilities.
  warnings.warn(


Sequence for CUT18449.1 changes from  to MPPRRKKTAAGAAAGGKAAAAAVGKAGFMPPKKPKKGKKKTPIMRYKRYIYKVLKQVRPELGISEKSTMIMNNFVAHNFQNIAKEASILAYYSKKRTITVDELKAAVAMVLPNLLADYANRDGEKAVSNFEGEASAKKSQGRKRGRGQQA
Sequence for CUT18450.1 changes from  to MAPKKKPSKLVGTVTKTRKVTETQTLKVSLTKGLKPEDQQTTTNKFEVSVTGKQSKTQPLIVSTNTNLVPKKEKEESPTTTLMVKKKRKNRKAGGEYKRYVYMVLKTVHPDMTVSSKAMMVMEGMMQDMFERLVTEAVRLVQYMKKATLTCREIQTAVMLVLPGELGKHAVSEGAKAITNYMAAVGSGNGGAA
Sequence for CUT18445.1 changes from  to MAPKSEKKPAEKKPVAEKPAAEEEKKSAPAPAAAEKKPAEKKPKAGKKLPASKGKEGEKKK
Sequence for CUT18446.1 changes from  to AEKKPKAGKKVPASKEGEKKKKRSKKSVETYKIYIFKVLKQVHPDIGISSKAMGIMNSFINDLFEKMAQESARLARYNKKNTITSREIQTSVRLVLPGELAKHAVSEGTKAVTKFTSA
Sequence for CUT18447.1 changes from  to MAPKAEKKPAAKKPAATPPPEEEKEVVPPPPAEKKPKAGKKLPAAKEGDAKKKKKAKKSIETYKIYIFKVLKQVHPDIGISSKAMGIMNSFINDIFEKLAQESARLARYNKKPTITSREIQTSVRLVLPGELAKHAVSEGTKAVTKFTSA
Sequence for CUT18448.1 changes from  to AATPPPEEEKEVVPPPAEKKPAEKKPKAGKKLPASKEGDAKKKKKSKKSIETYKIYIFKVLKQVHPDIGISSKAMGI

In [6]:
cs.data[cs.data["variant"] == "gH2B"]

,accession,type,variant_group,variant,doublet,gi,ncbi_gene_id,hgnc_gene_name,taxonomy_id,organism,phylum,class,taxonomy_group,info,references,sequence
CUT18449.1,CUT18449.1,H2B,gH2B,gH2B,,,,,1473204,Lilium davidii var. unicolor,Streptophyta,Magnoliopsida,,,26846354,MPPRRKKTAAGAAAGGKAAAAAVGKAGFMPPKKPKKGKKKTPIMRY...
CUT18450.1,CUT18450.1,H2B,gH2B,gH2B,,,,,1473204,Lilium davidii var. unicolor,Streptophyta,Magnoliopsida,,,26846354 32716939,MAPKKKPSKLVGTVTKTRKVTETQTLKVSLTKGLKPEDQQTTTNKF...
CUT18445.1,CUT18445.1,H2B,gH2B,gH2B,,,,,1473204,Lilium davidii var. unicolor,Streptophyta,Magnoliopsida,,,26846354,MAPKSEKKPAEKKPVAEKPAAEEEKKSAPAPAAAEKKPAEKKPKAG...
CUT18446.1,CUT18446.1,H2B,gH2B,gH2B,,,,,1473204,Lilium davidii var. unicolor,Streptophyta,Magnoliopsida,,,26846354,AEKKPKAGKKVPASKEGEKKKKRSKKSVETYKIYIFKVLKQVHPDI...
CUT18447.1,CUT18447.1,H2B,gH2B,gH2B,,,,,1473204,Lilium davidii var. unicolor,Streptophyta,Magnoliopsida,,,26846354,MAPKAEKKPAAKKPAATPPPEEEKEVVPPPPAEKKPKAGKKLPAAK...
CUT18448.1,CUT18448.1,H2B,gH2B,gH2B,,,,,1473204,Lilium davidii var. unicolor,Streptophyta,Magnoliopsida,,,26846354,AATPPPEEEKEVVPPPAEKKPAEKKPKAGKKLPASKEGDAKKKKKS...
CUT18451.1,CUT18451.1,H2B,gH2B,gH2B,,,,,1473204,Lilium davidii var. unicolor,Streptophyta,Magnoliopsida,,,26846354,MAPKAEKKPAAKKPAATPPPEEEKEVVPPPPAEKKPKAGKKLPAAK...
CUT18452.1,CUT18452.1,H2B,gH2B,gH2B,,,,,1473204,Lilium davidii var. unicolor,Streptophyta,Magnoliopsida,,,26846354,MAPKSEKKPAEKKPVAEKPAAEEEKKAAPAAAPAEKKAAEKKPKA


In [7]:
cs.save()

sequence  self                                                      
          other    MAPKAEKKPAAKKPAATPPPEEEKEVVPPPPAEKKPKAGKKLPAAK...
Name: CUT18447.1, dtype: object
sequence  self                                                      
          other    AATPPPEEEKEVVPPPAEKKPAEKKPKAGKKLPASKEGDAKKKKKS...
Name: CUT18448.1, dtype: object
sequence  self                                                      
          other    MAPKAEKKPAAKKPAATPPPEEEKEVVPPPPAEKKPKAGKKLPAAK...
Name: CUT18451.1, dtype: object
sequence  self                                                      
          other    MPPRRKKTAAGAAAGGKAAAAAVGKAGFMPPKKPKKGKKKTPIMRY...
Name: CUT18449.1, dtype: object
sequence  self                                                      
          other    MAPKSEKKPAEKKPVAEKPAAEEEKKSAPAPAAAEKKPAEKKPKAG...
Name: CUT18445.1, dtype: object
sequence  self                                                      
          other    AEKKPKAGKKVPASKEGEKKKKRSKKSVETYKIYIFKVLKQVHPDI...
Name: CUT184